In [1]:
import scipy.io
import numpy as np
import scipy as sp
import cupy as cp
import matplotlib.pyplot as plt
import os
from SWE_utils_cupy_pipelined import *
from scipy.signal import firwin, butter, buttord, freqz

from arrus.ops.us4r import *
from arrus.ops.imaging import *
from arrus.metadata import *
from arrus.devices.probe import *
from arrus.devices.us4r import Us4RDTO
from arrus.utils.imaging import *

In [3]:
focus = np.array([15, 20, 25, 30, 35])  #[mm]
f = 1.75 # funmber
ap = np.ceil(focus / (f * 0.298))
max_shift = ap / 2
print('Apertures:')
print(ap)
print('Max shifts:')
print(max_shift)

Apertures:
[29. 39. 48. 58. 68.]
Max shifts:
[14.5 19.5 24.  29.  34. ]


killall python
wyjmij/wloz wtyczke TB3

Petla: 

for i in $(seq 0 10); do python swe_procedure-test1a.py --pb_freq 4.0 --pb_focus 15 --rep $i ; done


for j in 15 16 17 18 19; do for i in $(seq 0 1); do python swe_procedure-test1a.py --pb_freq 4.0 --pb_focus $j --rep $i ; done ; done 


In [ ]:
size = [20, 21, 22]

for i in size:
    size_str = str(i)
    ! python testgen.py --size size_str --rep 3

## RF display

In [ ]:
%matplotlib widget

In [ ]:
# RF display
data = sp.io.loadmat("./Datasets/Bmode_manual.mat")
rf= data["rf_data"]
print(rf.shape)
data = np.squeeze(rf)
data = np.abs(data)

%matplotlib widget
plt.imshow(data[50], vmin=-1000, vmax=1000)

### B-mode display

In [ ]:
data = sp.io.loadmat("./Parametric_tests/Test_1/bmode_freq4.0_foc25.0.mat")
lri= data["data"]
print(lri.shape)
#lri = lri[5, :, :]
print(lri.shape)
lri = np.squeeze(lri)

if(1):
    #hri = np.mean(a=lri, axis=0)
    #hri = np.squeeze(lri[41, :, :])
    hri = lri

    # Envelope detection
    data = np.abs(hri)
    # Log compression
    data[data==0] = 10**-10
    data = 20 * np.log10(data)

    data_dim = data.shape
    if(data_dim[1] > data_dim[0] ):
        data = np.transpose(data, [1,0])

    dB_range = [20, 84] 
    norm = plt.Normalize(dB_range[0], dB_range[1], True)
    fig, ax0 = plt.subplots(1, 1, figsize=(8, 4))
    #%matplotlib widget
    ax0.imshow(data, cmap='gray', norm=norm)
    ax0.set_title('Bmode [dB]')

# Shear wave motion data

In [ ]:
data = sp.io.loadmat("./Parametric_tests/Test_1/swd_freq4.0_foc25.0.mat")
ddata_cpu= data["data"]
#print(ddata_cpu.dtype)
#ddata_cpu = np.real(ddata_cpu)
#ddata_cpu = np.abs(ddata_cpu)
print(ddata_cpu.shape)
print(ddata_cpu.dtype)

getHistogram(data=ddata_cpu, Vrange = [-0.5e-3, 0.5e-3])
DrawShearWaveFrames(data=ddata_cpu, Vrange=[-1e-3, 2e-3], frames=[1, 4, 7, 10, 13, 16])


In [ ]:
# Plot some profiles
%matplotlib widget
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
plt.rcParams.update({'font.size': 22})

In [ ]:
## Get spatio-temporal (sliced) frequency spectrum
depth_limits = [100, 110]
NFFT = [4096, 2048]
slice_in = ddata_cpu[depth_limits[0]:depth_limits[1], :, :]

def SW_spectralAnalysis_SpatioTemporal(data, depth_limits, NFFT):

    # Averaging data along axial dimension
    slice_in = np.squeeze(np.mean(data, axis=0))

    # Do FFT2
    slice_k = sp.fft.fft2(slice_in, s=NFFT)
    slice_k = sp.fft.fftshift(slice_k)

    # Normalize
    X = np.abs(slice_k)
    MaxX = np.max(X.flatten())
    Xn = X/MaxX
    
    # Log compression
    img = 20 * np.log10(Xn)
    return img

img = SW_spectralAnalysis_SpatioTemporal(data=slice_in, depth_limits=depth_limits, NFFT=NFFT)

## Results presentation ##
##  Spectral data
%matplotlib widget
plt.figure()
plt.clf()
plt.imshow(img+6.5, cmap='jet')
    


In [ ]:
# Obtain freq
x = 1280
fs = 5000
NFFT_f = NFFT[1]
xr = x - NFFT_f/2
f = xr * fs/NFFT_f
print(f)

In [ ]:
## Results presentation ##
## Motion data slice
px_size=0.2 #[mm]
pri = 0.2 #[ms]
tick_grid_size=[5, 5] #[ms, mm]

plt.figure()
plt.clf()
plt.imshow(slice_in, cmap='gray')
plt.title('Spatio-temporal motion data slice')
plt.xlabel("Slow-time [ms]")
plt.ylabel("x [mm]")

# Assign ticks
dim = slice_in.shape
y_grid = tick_grid_size[1]  #[mm]
x_grid = tick_grid_size[0]  #[mm]

m = int(dim[0]//2)
a = np.arange(m, 0, -int(np.ceil(x_grid / px_size)))
b = np.arange(m, dim[0]-1, int(np.ceil(x_grid / px_size)))
yticks = np.concatenate((a[1:], b))
yticks_labels = (yticks - m) * px_size
yticks_labels = [str(int(x)) for x in yticks_labels]
plt.yticks(yticks, yticks_labels)

xticks  = np.arange(0, dim[1]-1, int(np.ceil(x_grid / pri))) 
xticks_labels = xticks * pri
xticks_labels = [str(int(x)) for x in xticks_labels]
plt.xticks(xticks, xticks_labels)


In [ ]:
## Get temporal frequency spectrum

data = ddata_cpu[85:125, 100:150, :]
print(data.shape)
Fs = 1/200e-6
NFFT = 2048

def SW_spectralAnalysis_SpatioTemporal(data, Fs, NFFT):

    # Apply window function
    N = data.shape[2]
    w = sp.signal.windows.tukey(N, alpha=0.2, sym=True)
    data_w = data * w

    # Do fft
    Hy = sp.fft.fft(data_w, NFFT)
    P2 = np.abs(Hy/NFFT)
    P1 = P2[:, :, 1:NFFT//2+1]
    P1[:, :, 1:-1] = 2*P1[:, :, 1:-1]

    # Average
    Pmean = np.mean(P1, axis=0)
    Pmean = np.mean(Pmean, axis=0)

    #Plot
    x_f = np.arange(0, NFFT//2, 1)
    x_f = Fs*x_f/NFFT

    M = np.max(Pmean)
    Xn = Pmean/M
    Pmean_dB = 20 * np.log10(Xn)
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    ax.plot(x_f, Pmean_dB+3.5)
    return

SW_spectralAnalysis_SpatioTemporal(data=data, Fs=Fs, NFFT=NFFT)


## Post shear wave motion filtering data

In [ ]:
data = sp.io.loadmat("./Datasets/swdf.mat")
ddata_cpu= data["data"]
print(ddata_cpu.shape)
print(ddata_cpu.dtype)

DrawShearWaveFrames(data=np.squeeze(ddata_cpu[0, ...]), Vrange=[-1e-3, 2e-3], frames=[1, 8, 15, 20, 26, 32])

In [ ]:
# Plot some profiles
%matplotlib widget
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(np.squeeze(ddata_cpu[0, 100, 150-15, :]))
ax.plot(np.squeeze(ddata_cpu[0, 100, 150+15, :]))

In [ ]:
## Get energy map
ddataE = np.squeeze(ddata_cpu[0, 10:-20, ...])

# Get avarage and delete it
m = np.mean(ddataE, axis=2)
ddataE = np.transpose(ddataE, [2,0,1])
ddataE = np.subtract(ddataE, m)
ddataE = np.transpose(ddataE, [1,2,0])
E = np.power(ddataE, 2)
E = np.squeeze(np.sum(E, 2))
E = np.sqrt(E)

norm = plt.Normalize(0, 0.002, True)
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
print(E.shape)
ax.imshow(E, cmap='jet', norm=norm)

        

In [ ]:
# Display energy map


## SWS data

In [ ]:
data = sp.io.loadmat("./Parametric_tests/Test_1/sws_freq4.0_foc25.0_rep0.mat")
SWV_cpu= data["data"]
print(SWV_cpu.shape)
print(SWV_cpu.dtype)


In [ ]:
#Visualize the results (pre-compounding)
SWS_cpu = np.squeeze(SWV_cpu[0, ...])
SWS_r_cpu = np.squeeze(SWV_cpu[1, ...])
print()
norm_sws = plt.Normalize(0, 5, True)
norm_r   = plt.Normalize(0, 1, True)

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
axs[0,0].imshow(np.squeeze(SWS_cpu[0,:,:]), cmap='jet', norm=norm_sws)
axs[0,0].set_title('SWS_LR [m/s]')

axs[1,0].imshow(np.squeeze(SWS_r_cpu[0,:,:]), cmap='jet', norm=norm_r)
axs[1,0].set_title('r_LR')

axs[0,1].imshow(np.squeeze(SWS_cpu[1,:,:]), cmap='jet', norm=norm_sws)
axs[0,1].set_title('SWS_RL [m/s]')

axs[1,1].imshow(np.squeeze(SWS_r_cpu[1,:,:]), cmap='jet', norm=norm_r)
axs[1,1].set_title('r_RL')

In [ ]:
#Visualize the results (post-compounding)
SWS_cpu = np.squeeze(SWV_cpu[0, ...])
SWS_r_cpu = np.squeeze(SWV_cpu[1, ...])
print()
sws_range = [0.5, 4.0]
norm_sws = plt.Normalize(0, 5.0, True)
norm_r   = plt.Normalize(0, 2, True)

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(12, 8))
axs[0].imshow(np.squeeze(SWS_cpu), cmap='jet', norm=norm_sws)
axs[0].set_title('SWS_LR [m/s]')

axs[1].imshow(np.squeeze(SWS_r_cpu), cmap='jet', norm=norm_r)
axs[1].set_title('r_LR')


# Post-processing

## Image compounding

In [ ]:
# Visualize results
SWSc_cpu  = SWSc.get()
SWS = np.squeeze(SWSc_cpu[0])
r   = np.squeeze(SWSc_cpu[1])
norm_r   = plt.Normalize(0, 2, True)
norm_sws = plt.Normalize(swse_SWV_range[0], swse_SWV_range[1], True)
norm_sws = plt.Normalize(0, 5, True)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8, 6))
axs[0].imshow(np.squeeze(SWS), cmap='jet', norm=norm_sws)
axs[0].set_title('SWSc [m/s]')

axs[1].imshow(np.squeeze(r), cmap='jet', norm=norm_r)
axs[1].set_title('r [m/s]')

## Image filtering

In [ ]:
# Visualize results
SWSc_cpu  = SWSc.get()
SWS = np.squeeze(SWSc_cpu[0])
r   = np.squeeze(SWSc_cpu[1])
norm_r   = plt.Normalize(0, 2, True)
norm_sws = plt.Normalize(swse_SWV_range[0], swse_SWV_range[1], True)
norm_sws = plt.Normalize(0, 5, True)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8, 6))
axs[0].imshow(np.squeeze(SWS), cmap='jet', norm=norm_sws)
axs[0].set_title('SWSc [m/s]')

axs[1].imshow(np.squeeze(r), cmap='jet', norm=norm_r)
axs[1].set_title('r [m/s]')

## Data presentation

In [ ]:
# Get data from GPU
SWSc_cpu  = SWSc.get()
SWS = np.squeeze(SWSc_cpu[0])
r   = np.squeeze(SWSc_cpu[1])

In [ ]:
# Visualize SWS map
plotSWSmap(data=SWS[swse_z_clip[0]:-swse_z_clip[1], swse_d//2:-swse_d//2], px_size=px_size, tick_grid_size=[5, 5], sws_disp_range=[0, 5])
plotSWSmap(data=r[swse_z_clip[0]:-swse_z_clip[1], swse_d//2:-swse_d//2], px_size=px_size, tick_grid_size=[5, 5], sws_disp_range=[0, 2])